In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from bokeh.io import curdoc, output_notebook
from bokeh.plotting import figure, show, curdoc
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper, Slider, NumeralTickFormatter
from bokeh.palettes import Spectral6
from bokeh.layouts import row, layout
import plotly.express as px

In [2]:
df = pd.read_csv("store.csv", encoding="ISO-8859-1")

In [3]:
df.head()

,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub-Category,Sales,Quantity,Discount,Profit
0,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,261.9600,2,0.00,41.9136
1,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,731.9400,3,0.00,219.5820
2,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,14.6200,2,0.00,6.8714
3,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,957.5775,5,0.45,-383.0310
4,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,22.3680,2,0.20,2.5164


In [4]:
df.isnull().sum()
# Check for any NaN values in the whole DataFrame
df.isnull().values.any()

# Check if all values are NaN in the DataFrame
df.isnull().values.all()

False

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Ship Mode     9994 non-null   object 
 1   Segment       9994 non-null   object 
 2   Country       9994 non-null   object 
 3   City          9994 non-null   object 
 4   State         9994 non-null   object 
 5   Postal Code   9994 non-null   int64  
 6   Region        9994 non-null   object 
 7   Category      9994 non-null   object 
 8   Sub-Category  9994 non-null   object 
 9   Sales         9994 non-null   float64
 10  Quantity      9994 non-null   int64  
 11  Discount      9994 non-null   float64
 12  Profit        9994 non-null   float64
dtypes: float64(3), int64(2), object(8)
memory usage: 1015.1+ KB


In [6]:
import streamlit as st
import plotly.express as px
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

st.set_page_config(page_title="Superstore!!!", page_icon=":bar_chart:",layout="wide")

st.title(" :bar_chart: Sample SuperStore EDA")

2023-11-18 00:56:53.653 
  command:

    streamlit run /Users/annageorgieva/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [8]:
st.markdown('<style>div.block-container{padding-top:1rem;}</style>',unsafe_allow_html=True)

fl = st.file_uploader(":file_folder: Upload a file",type=(["csv","txt","xlsx","xls"]))
if fl is not None:
    filename = fl.name
    st.write(filename)
    df = pd.read_csv(filename, encoding = "ISO-8859-1")
else:
    os.chdir("/Users/annageorgieva/Documents/GitHub/BigData-DV")
    df = pd.read_csv("store.csv", encoding="ISO-8859-1")

In [11]:
col1, col2 = st.columns((2))
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

# Getting the min and max date 
startDate = pd.to_datetime(df["InvoiceDate"]).min()
endDate = pd.to_datetime(df["InvoiceDate"]).max()

with col1:
    date1 = pd.to_datetime(st.date_input("Start Date", startDate))

with col2:
    date2 = pd.to_datetime(st.date_input("End Date", endDate))
    df = df[(df["InvoiceDate"] >= date1) & (df["InvoiceDate"] <= date2)].copy()

In [12]:
st.sidebar.header("Choose your filter: ")
# Create for Region
region = st.sidebar.multiselect("Pick your Region", df["Region"].unique())
if not region:
    df2 = df.copy()
else:
    df2 = df[df["Region"].isin(region)]

In [14]:
# Create for State
state = st.sidebar.multiselect("Pick the State", df2["State"].unique())
if not state:
    df3 = df2.copy()
else:
    df3 = df2[df2["State"].isin(state)]

# Create for City
city = st.sidebar.multiselect("Pick the City",df3["City"].unique())

if not region and not state and not city:
    filtered_df = df
elif not state and not city:
    filtered_df = df[df["Region"].isin(region)]
elif not region and not city:
    filtered_df = df[df["State"].isin(state)]
elif state and city:
    filtered_df = df3[df["State"].isin(state) & df3["City"].isin(city)]
elif region and city:
    filtered_df = df3[df["Region"].isin(region) & df3["City"].isin(city)]
elif region and state:
    filtered_df = df3[df["Region"].isin(region) & df3["State"].isin(state)]
elif city:
    filtered_df = df3[df3["City"].isin(city)]
else:
    filtered_df = df3[df3["Region"].isin(region) & df3["State"].isin(state) & df3["City"].isin(city)]

In [17]:
category_df = filtered_df.groupby(by = ["Product"], as_index = False)["TotalSales"].sum()

In [19]:
with col1:
    st.subheader("Product wise Sales")
    # Ensure that TotalSales is a float and handle any potential conversion errors
    category_df["TotalSales"] = pd.to_numeric(category_df["TotalSales"], errors='coerce')
    # Now, you can safely format the numbers
    fig = px.bar(category_df, x="Product", y="TotalSales",
                 text=['${:,.2f}'.format(x) for x in category_df["TotalSales"]],
                 template="seaborn")
    st.plotly_chart(fig, use_container_width=True, height=200)

In [20]:
with col2:
    st.subheader("Region wise Sales")
    fig = px.pie(filtered_df, values = "TotalSales", names = "Region", hole = 0.5)
    fig.update_traces(text = filtered_df["Region"], textposition = "outside")
    st.plotly_chart(fig,use_container_width=True)

In [21]:
cl1, cl2 = st.columns((2))
with cl1:
    with st.expander("Category_ViewData"):
        st.write(category_df.style.background_gradient(cmap="Blues"))
        csv = category_df.to_csv(index = False).encode('utf-8')
        st.download_button("Download Data", data = csv, file_name = "Category.csv", mime = "text/csv",
                            help = 'Click here to download the data as a CSV file')

In [23]:
with cl2:
    with st.expander("Region_ViewData"):
        region = filtered_df.groupby(by = "Region", as_index = False)["TotalSales"].sum()
        st.write(region.style.background_gradient(cmap="Oranges"))
        csv = region.to_csv(index = False).encode('utf-8')
        st.download_button("Download Data", data = csv, file_name = "Region.csv", mime = "text/csv",
                        help = 'Click here to download the data as a CSV file')

In [25]:
filtered_df["month_year"] = filtered_df["InvoiceDate"].dt.to_period("M")
st.subheader('Time Series Analysis')

DeltaGenerator()

In [26]:
linechart = pd.DataFrame(filtered_df.groupby(filtered_df["month_year"].dt.strftime("%Y : %b"))["TotalSales"].sum()).reset_index()
fig2 = px.line(linechart, x = "month_year", y="TotalSales", labels = {"TotalSales": "Amount"},height=500, width = 1000,template="gridon")
st.plotly_chart(fig2,use_container_width=True)

DeltaGenerator()

In [27]:
with st.expander("View Data of TimeSeries:"):
    st.write(linechart.T.style.background_gradient(cmap="Blues"))
    csv = linechart.to_csv(index=False).encode("utf-8")
    st.download_button('Download Data', data = csv, file_name = "TimeSeries.csv", mime ='text/csv')

In [31]:
# Create a treem based on Region, category, sub-Category
st.subheader("Hierarchical view of Sales using TreeMap")
fig3 = px.treemap(filtered_df, path = ["Region","Product","Product"], values = "TotalSales",hover_data = ["TotalSales"],
                  color = "Product")
fig3.update_layout(width = 800, height = 650)
st.plotly_chart(fig3, use_container_width=True)

ValueError: The column label 'Product' is not unique.